<a href="https://colab.research.google.com/github/Amir-Rastkhadiv/RMHC-Run-Raiser-Agent/blob/main/notebooks/Capstone_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pydantic



In [2]:
from __future__ import annotations

from enum import Enum
from typing import List, Optional, Dict, Any
from pydantic import BaseModel
from datetime import datetime, timezone
from uuid import uuid4
from textwrap import indent


# =========================
# SCHEMAS & ENUMS
# =========================

class Platform(str, Enum):
    LINKEDIN = "linkedin"
    STRAVA = "strava"
    INTERNAL = "internal"


class Tone(str, Enum):
    PROFESSIONAL = "professional"
    MOTIVATIONAL = "motivational"
    CASUAL = "casual"


class JudgeDecision(str, Enum):
    APPROVE = "APPROVE"
    REVISE = "REVISE"
    REJECT = "REJECT"


class ActivitySummary(BaseModel):
    date: str
    distance_km: float
    duration_min: int
    pace_min_per_km: float
    event_name: str
    route_name: str


class DonationDetail(BaseModel):
    donor_name: str
    amount_usd: float
    message: Optional[str] = None


class FundraisingSummary(BaseModel):
    total_raised_usd: float
    target_amount_usd: float
    percent_to_goal: float
    recent_donations: List[DonationDetail]


class PostRequest(BaseModel):
    target_platform: Platform
    tone: Tone
    objective: str
    audience: str
    call_to_action_hint: str


class PostCandidate(BaseModel):
    candidate_id: str
    platform: Platform
    text: str
    rationale: str
    risk_flags: List[str]


class JudgeFeedback(BaseModel):
    candidate_id: str
    score_0_100: int
    decision: JudgeDecision
    reasons: str
    required_edits: Optional[str] = None


class TrajectoryStep(BaseModel):
    step_number: int
    step_name: str
    timestamp: str
    description: str
    tools_used: List[str]
    notes: Optional[Dict[str, Any]] = None


class TrajectoryLog(BaseModel):
    session_id: str
    steps: List[TrajectoryStep]
    final_decision_summary: str


# =========================
# MEMORY MODELS
# =========================

class StatisticalMemory(BaseModel):
    total_lifetime_raised_usd: float = 0.0
    total_lifetime_km_run: float = 0.0
    last_update_date: str = "N/A"


class EpisodicMemory(BaseModel):
    past_successful_posts: List[str] = []


class FullMemory(BaseModel):
    statistical: StatisticalMemory = StatisticalMemory()
    episodic: EpisodicMemory = EpisodicMemory()


def get_memory_state() -> FullMemory:
    """
    Return pre-populated memory (simulated).
    Inspired by the real fundraising: ~£507+ raised.
    """
    return FullMemory(
        statistical=StatisticalMemory(
            total_lifetime_raised_usd=507.15,
            total_lifetime_km_run=42.2,
            last_update_date=datetime.now(timezone.utc).date().isoformat(),
        ),
        episodic=EpisodicMemory(
            past_successful_posts=[
                "Huge thank you to everyone who helped us cross the £500 line for RMHC!"
            ]
        ),
    )


# =========================
# TOOLS (SIMULATED)
# =========================

# TYPE: Retrieving Information (R)
def get_activity_summary() -> ActivitySummary:
    """Return a simulated running activity summary."""
    return ActivitySummary(
        date=datetime.now(timezone.utc).date().isoformat(),
        distance_km=10.0,
        duration_min=55,
        pace_min_per_km=5.5,
        event_name="Southend Seafront 10K",
        route_name="Black Friday Coastal Loop",
    )


# TYPE: Retrieving Information (R)
def get_fundraising_summary() -> FundraisingSummary:
    """Return a simulated fundraising summary (~£507 milestone)."""
    total = 507.15
    target = 500.0
    percent = round((total / target) * 100, 1)

    recent = [
        DonationDetail(donor_name="Anonymous", amount_usd=25.0, message="Keep going!"),
        DonationDetail(donor_name="Local Guest", amount_usd=15.0, message="For RMHC families."),
    ]

    return FundraisingSummary(
        total_raised_usd=total,
        target_amount_usd=target,
        percent_to_goal=percent,
        recent_donations=recent,
    )


# TYPE: Executing Action (A) – content generation (simulated)
def generate_post_candidates(
    post_request: PostRequest,
    activity: ActivitySummary,
    fundraising: FundraisingSummary,
) -> List[PostCandidate]:
    """
    Simulate three candidate posts instead of actually calling Gemini.
    """
    base_text = (
        f"We’ve just passed £{fundraising.total_raised_usd:.2f} for Ronald McDonald House Charities, "
        f"after a {activity.distance_km:.1f} km run along {activity.route_name}."
    )

    candidates: List[PostCandidate] = []

    candidates.append(
        PostCandidate(
            candidate_id="c1",
            platform=post_request.target_platform,
            text=(
                f"{base_text} Thank you to all {len(fundraising.recent_donations)} recent donors – "
                f"your support keeps families close to their children in hospital. "
                f"{post_request.call_to_action_hint}"
            ),
            rationale="Balanced gratitude, mentions impact for RMHC families.",
            risk_flags=[],
        )
    )

    candidates.append(
        PostCandidate(
            candidate_id="c2",
            platform=post_request.target_platform,
            text=(
                f"{base_text} We’ve smashed our initial £{fundraising.target_amount_usd:.0f} goal, "
                f"but every extra pound helps another family stay near the care they need."
            ),
            rationale="Focuses on milestone and continued need.",
            risk_flags=[],
        )
    )

    candidates.append(
        PostCandidate(
            candidate_id="c3",
            platform=post_request.target_platform,
            text=(
                f"{base_text} If you can, please consider donating or sharing this campaign today "
                f"so RMHC can support even more families."
            ),
            rationale="Gentle call-to-action with clear purpose.",
            risk_flags=[],
        )
    )

    return candidates


# TYPE: Executing Action (A) – LLM-as-a-Judge (simulated)
def judge_post_quality(candidate: PostCandidate, platform: Platform) -> JudgeFeedback:
    """
    Simulated judge.

    In the final agent, this is where Gemini would be called.
    """
    base_score = 80
    if "smashed" in candidate.text.lower():
        base_score += 5
    if "please consider donating" in candidate.text.lower():
        base_score += 5

    score = min(base_score, 95)
    decision = JudgeDecision.APPROVE
    reasons = (
        f"Tone appropriate for {platform.value}; clear gratitude and impact; "
        "no obvious sensitivity issues."
    )

    return JudgeFeedback(
        candidate_id=candidate.candidate_id,
        score_0_100=score,
        decision=decision,
        reasons=reasons,
        required_edits=None,
    )


# TYPE: Executing Action (A)
def simulate_publish_post(final_post: PostCandidate) -> str:
    """Return a string describing the simulated publish."""
    return f"[SIMULATED PUBLISH] Platform={final_post.platform.value} | Text='{final_post.text[:120]}...'"


# TYPE: Executing Action (A)
def update_memory_state(memory: FullMemory, final_post: PostCandidate) -> str:
    """Pretend to update memory (no persistence in this demo)."""
    _ = memory
    _ = final_post
    return "[MEMORY UPDATED] Added latest post to episodic history."


# =========================
# AGENT (7-STEP ORCHESTRATOR)
# =========================

SYSTEM_INSTRUCTION = """
You are the RMHC Run-Raiser Agent.

Mission:
- Help a fundraiser for Ronald McDonald House Charities communicate milestones
  safely and effectively on LinkedIn, Strava-style feeds, and internal channels.
- Always protect the RMHC brand (empathy, accuracy, sensitivity).

You follow a mandatory 7-STEP TRAJECTORY:
1) Retrieve memory
2) Retrieve latest activity + fundraising data
3) Plan communication strategy (persona, tone, objective)
4) Generate post candidates
5) Quality Gate (LLM-as-a-Judge)
6) Execute action (simulate publish)
7) Update memory + log the trajectory
"""


class RMHCRunRaiserAgent:
    """Level 2 orchestrator with tools, memory, and observability."""

    def __init__(self) -> None:
        self.system_instruction = SYSTEM_INSTRUCTION

    def _now_iso(self) -> str:
        """Return current UTC time as ISO string (timezone-aware)."""
        return datetime.now(timezone.utc).isoformat()

    def _add_step(
        self,
        steps: List[TrajectoryStep],
        step_number: int,
        step_name: str,
        description: str,
        tools_used: Optional[List[str]] = None,
        notes: Optional[Dict[str, Any]] = None,
    ) -> None:
        step = TrajectoryStep(
            step_number=step_number,
            step_name=step_name,
            timestamp=self._now_iso(),
            description=description,
            tools_used=tools_used or [],
            notes=notes or {},
        )
        steps.append(step)

    def run(self, post_request: PostRequest) -> Dict[str, Any]:
        """Run full 7-step trajectory for one PostRequest."""
        steps: List[TrajectoryStep] = []
        session_id = str(uuid4())

        # Step 1: memory
        memory = get_memory_state()
        self._add_step(
            steps, 1, "Retrieve Memory",
            "Loaded statistical and episodic memory.",
            tools_used=["get_memory_state"],
        )

        # Step 2: data
        activity = get_activity_summary()
        fundraising = get_fundraising_summary()
        self._add_step(
            steps, 2, "Retrieve Data",
            "Fetched latest activity and fundraising stats.",
            tools_used=["get_activity_summary", "get_fundraising_summary"],
            notes={
                "distance_km": activity.distance_km,
                "total_raised_usd": fundraising.total_raised_usd,
                "percent_to_goal": fundraising.percent_to_goal,
            },
        )

        # Step 3: plan
        self._add_step(
            steps, 3, "Plan Communication Strategy",
            "Using PostRequest to select persona and objective.",
            notes={
                "target_platform": post_request.target_platform.value,
                "tone": post_request.tone.value,
                "objective": post_request.objective,
                "audience": post_request.audience,
            },
        )

        # Step 4: generate candidates
        candidates = generate_post_candidates(post_request, activity, fundraising)
        self._add_step(
            steps, 4, "Generate Candidates",
            f"Generated {len(candidates)} post candidates (simulated Gemini).",
            tools_used=["generate_post_candidates"],
            notes={"candidate_ids": [c.candidate_id for c in candidates]},
        )

        # Step 5: judge
        best_candidate = None
        best_feedback = None
        best_score = -1
        for c in candidates:
            fb = judge_post_quality(c, post_request.target_platform)
            if fb.score_0_100 > best_score:
                best_score = fb.score_0_100
                best_candidate = c
                best_feedback = fb

        self._add_step(
            steps, 5, "Quality Gate (Judge)",
            "Evaluated candidates and selected the top-scoring safe option.",
            tools_used=["judge_post_quality"],
            notes={
                "selected_candidate_id": best_candidate.candidate_id,
                "score_0_100": best_feedback.score_0_100,
                "decision": best_feedback.decision,
            },
        )

        # Step 6: simulate publish
        publish_result = simulate_publish_post(best_candidate)
        self._add_step(
            steps, 6, "Simulate Publish",
            "Simulated publishing the approved post.",
            tools_used=["simulate_publish_post"],
            notes={"publish_result": publish_result},
        )

        # Step 7: update memory + finalise
        update_memory_state(memory, best_candidate)
        final_summary = (
            f"Published a {post_request.target_platform.value} post with objective "
            f"'{post_request.objective}'. Judge score: {best_feedback.score_0_100}."
        )
        self._add_step(
            steps, 7, "Update Memory & Finalise",
            "Updated memory (simulated) and finalised trajectory.",
            tools_used=["update_memory_state"],
            notes={"final_decision_summary": final_summary},
        )

        trajectory_log = TrajectoryLog(
            session_id=session_id,
            steps=steps,
            final_decision_summary=final_summary,
        )

        return {
            "final_post": best_candidate,
            "judge_feedback": best_feedback,
            "trajectory_log": trajectory_log,
        }


# =========================
# DEMO RUNNER
# =========================

def pretty_print_trajectory(log: TrajectoryLog) -> None:
    print("\n==================== TRAJECTORY LOG ====================")
    print(f"Session ID: {log.session_id}")
    print("--------------------------------------------------------")
    for step in log.steps:
        print(f"[Step {step.step_number}] {step.step_name} @ {step.timestamp}")
        print(indent(step.description, "  "))
        if step.tools_used:
            print(f"  Tools used: {', '.join(step.tools_used)}")
        if step.notes:
            print("  Notes:")
            for k, v in (step.notes or {}).items():
                print(f"    - {k}: {v}")
        print("--------------------------------------------------------")
    print("FINAL DECISION SUMMARY:")
    print(indent(log.final_decision_summary, "  "))
    print("========================================================\n")


def run_demo():
    post_request = PostRequest(
        target_platform=Platform.LINKEDIN,
        tone=Tone.PROFESSIONAL,
        objective="Celebrate hitting the £500+ fundraising milestone and encourage further support.",
        audience="Corporate partners and professional network",
        call_to_action_hint="Invite colleagues and partners to donate or share the campaign.",
    )

    agent = RMHCRunRaiserAgent()
    result = agent.run(post_request)

    final_post = result["final_post"]
    judge_feedback = result["judge_feedback"]
    trajectory_log = result["trajectory_log"]

    print("\n==================== FINAL APPROVED POST ====================")
    print(f"Platform: {final_post.platform.value}")
    print("Text:")
    print(indent(final_post.text, "  "))
    print("\nRationale:")
    print(indent(final_post.rationale, "  "))
    print("\nJudge Feedback:")
    print(f"  Score:    {judge_feedback.score_0_100}")
    print(f"  Decision: {judge_feedback.decision}")
    print(f"  Reasons:  {judge_feedback.reasons}")
    if judge_feedback.required_edits:
        print(f"  Required edits: {judge_feedback.required_edits}")
    print("=============================================================")

    pretty_print_trajectory(trajectory_log)


In [3]:
run_demo()



==================== FINAL APPROVED POST ====================
Platform: linkedin
Text:
  We’ve just passed £507.15 for Ronald McDonald House Charities, after a 10.0 km run along Black Friday Coastal Loop. We’ve smashed our initial £500 goal, but every extra pound helps another family stay near the care they need.

Rationale:
  Focuses on milestone and continued need.

Judge Feedback:
  Score:    85
  Decision: JudgeDecision.APPROVE
  Reasons:  Tone appropriate for linkedin; clear gratitude and impact; no obvious sensitivity issues.

==================== TRAJECTORY LOG ====================
Session ID: ca211de2-14b8-45a2-bb57-9e50ad6bcd5c
--------------------------------------------------------
[Step 1] Retrieve Memory @ 2025-11-30T06:03:20.537895+00:00
  Loaded statistical and episodic memory.
  Tools used: get_memory_state
--------------------------------------------------------
[Step 2] Retrieve Data @ 2025-11-30T06:03:20.537958+00:00
  Fetched latest activity and fundraising stats.
